# OVK-LIWC correlation

Test the correlation of the columns in the file ovk-liwc-wide.csv

In [ ]:
import csv
import re
import sys

INFILE = "ovk-liwc-wide.csv"

def readFile(inFileName):
    try:
        table = []
        inFile = open(inFileName,"r")
        csvreader = csv.DictReader(inFile,delimiter=",")
        for row in csvreader: table.append(row)
        inFile.close()
        return(table)
    except Exception as e:
        sys.exit("error processing file "+inFileName+": "+str(e))

table = readFile(INFILE)

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

def plot(x,y,title=""):
    fig = plt.figure()
    plt.scatter(x,y)
    plt.title(title)
    plt.show()
    plt.savefig("ovk-liwc-correlation.png")

First, determine the correlation between the CESD score and the MHC score. The expectation is that this would be negative, since CESD measures depression (the higher, the more depression signals) and MHC measures mental health (the higher, the better). The expection proves to be correct. The correlation factor is -0.34 and the negative correlation is identifyable from the plot.

In [ ]:
import numpy as np

cesd = [float(x["cesdT1"])-float(x["cesdT0"]) for x in table if x["cesdT1"] != "NA" and x["mhcT1"] != "NA" ]
mhc = [float(y["mhcT1"])-float(y["mhcT0"]) for y in table if y["cesdT1"] != "NA" and y["mhcT1"] != "NA"]

np.corrcoef(cesd,mhc)[0][1]

In [ ]:
plot(cesd,mhc,"change in CESD (x) vs MHC (y); "+str(len(cesd))+" clients")

Next, we determine the correlation between the change of CESD and MHC rates with any of the other features. We discard the features from T1 because we are interested in determing characteristics of patients from T0 data. We find that three features are weakly correlated with both CESD change and MHC change: CLIENTIDT0, sexualT0 and assentT0. The absolute correlation factors of these fetaures is about 0.2.

In [ ]:
MINCOR = 0.15

def numeric(table,key):
    for row in table:
        if row[key] != "NA":
            try: float(row[key])
            except: return(False)
    return(True)

def computeCorrelations(targetName):
    correlations = {}
    for key in table[0]:
        if numeric(table,key):
            target = [float(x[targetName+"T1"])-float(x[targetName+"T0"]) \
                      for x in table if x[targetName+"T1"] != "NA" and x[key] != "NA"]
            y = [float(y[key]) for y in table if y[targetName+"T1"] != "NA" and y[key] != "NA"]
            if len(set(y)) > 1: correlations[key] = np.corrcoef(target,y)[0][1]
    for key in sorted(correlations,key=correlations.__getitem__,reverse=True): 
        if abs(correlations[key]) >= MINCOR and not re.search("T1$",key):
            print(correlations[key],key)
            
computeCorrelations("cesd")
print("\n***\n")
computeCorrelations("mhc")

In [ ]:
def plotFeature(table,measureName,featureName,title="",sb=1): 
    measure = [float(x[measureName+"T1"])-float(x[measureName+"T0"]) \
               for x in table if x[measureName+"T1"] != "NA" and x[featureName] != "NA"]
    feature = [float(x[featureName]) \
               for x in table if x[measureName+"T1"] != "NA" and x[featureName] != "NA"]
    corr = np.corrcoef(measure,feature)[0][1]
    plot(measure,feature,measureName+" vs "+featureName+" ("+str(round(corr,2))+")")

plotFeature(table,"mhc","assentT0","CESD vs assentT0",1)
plotFeature(table,"mhc","sexualT0","CESD vs sexualT0",2)
plotFeature(table,"mhc","avgWordLenCliT0","CESD vs avgWordLenCliT0",3)

Next, we check the relation between counselors and the thepary result. For this purpose, we need to convert the counselor ids to numbers. There are several ways to do this (different orders). We do not know what works best so we check all possible orders and select the best result.

In [ ]:
from itertools import permutations

CESDT0 = "cesdT0"
CESDT1 = "cesdT1"
MHCT0 = "mhcT0"
MHCT1 = "mhcT1"
COUNSELOR = "counselor"
NA = "NA"

counselorList = [x[COUNSELOR] for x in table]
counselors = sorted(list(set(counselorList)))
cesd = [float(x[CESDT1])-float(x[CESDT0]) for x in table if x[CESDT1] != NA ]
mhc = [float(x[MHCT1])-float(x[MHCT0]) for x in table if x[MHCT1] != NA ]

bestShuffle = []
bestCorr = 0
for shuffle in permutations([x for x in range(0,len(counselors))]):
    shuffled = {counselors[i]:shuffle[i] for i in range(0,len(counselors))}
    counselorsShuffled = [shuffled[x[COUNSELOR]] for x in table if x[MHCT1] != NA]
    corr = np.corrcoef(mhc,counselorsShuffled)[0][1]
    if abs(corr) > bestCorr:
        bestCorr = corr
        bestShuffle = shuffle
print(bestCorr,bestShuffle)

In [ ]:
shuffled = {counselors[i]:bestShuffle[i] for i in range(0,len(counselors))}
counselorsShuffled = [shuffled[x[COUNSELOR]] for x in table if x[MHCT1] != NA]
corr = np.corrcoef(mhc,counselorsShuffled)[0][1]
plot(mhc,counselorsShuffled)
corr

Even the strongest correlated features are only weakly correlated (0.2). The small data size could be a cause of this. We could explore two directions for improving this:

1. Increase the size of the current dataset
2. Look for other features